# 개인프로젝트
###  #01.프로젝트 개요

## 관심사
- 관심있는 브랜드의 남성 스포츠 용품에 대한 정보 수집 : 미즈노, 아디다스 


        미즈노 남성 스포츠용품 웹페이지 url : https://kor.mizuno.com/kr-kr/044/?start=0&sz=12

        


- 미즈노 크롤링 하나 데이터프레임+ 아디다스 크롤링 데이터 프레임 하나로  합치기 


In [74]:
# 필요한 모듈 참조
import requests
from bs4 import BeautifulSoup
from pandas import DataFrame    

import chromedriver_autoinstaller
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import time


#### #02.웹 페이지 코드 수집 - 미즈노부터 ㄱㄱ 


미즈노는 각 페이지 별로 상품 수집해서 그 전에는 이미지링크 + 상품 상세보기 링크 + 상품이름 -> 링크를 타고 들어가자 
그다음에 정가, 할인가 + 리뷰 수 

In [75]:


## chromedriver를 통해 크롬브라우저 가동
chromedriver_autoinstaller.install()
driver = webdriver.Chrome()
driver.implicitly_wait(5)

In [76]:

## 수집할 컨텐츠가 있는 웹페이지의 주소 

## https://kor.mizuno.com/kr-kr/044/  
## https://kor.mizuno.com/kr-kr/044/?start=0&sz=12

## 페이지마다 url주소의 start 값이 12씩 늘어남
mizuno_num =[]
for i in range(0,5) :
    mizuno_num.append(i*12)
print(mizuno_num)



[0, 12, 24, 36, 48]


#### #03. 응답결과로부터 내용 추출하기

In [77]:
## 결과를 수집할 리스트 지정 (딕셔너리를 원소로 갖는 리스트)
mizuno_result =[]
# 상품리스트가 나온 페이지에서 
# 링크 + 상품이름 + 이미지링크 + 상품 상세보기 추출
for number in mizuno_num :
    url = "https://kor.mizuno.com/kr-kr/044/?start=%d&sz=12" % number
    # url 접속 
    driver.get(url)
    time.sleep(1)

    ## 전체 HTML 파일
    soup = BeautifulSoup(driver.page_source)
    mizuno_list = soup.select("#search-result-items .product-tile")
    print("페이지: %d" %((number/12)+1))
    print("-" * 50)
    result_dict = {}
    for mizuno in mizuno_list:
        # 상품이름
        mizuno_nameEl= mizuno.select(".name-link")
        mizuno_name = mizuno_nameEl[0].text.strip()
        # print(mizuno_name)

        ## 이미지 링크 
        mizuno_imageEl = mizuno.select(".thumb-link > img")
        mizuno_image = mizuno_imageEl[0].attrs['src']
        # print(mizuno_image)
        ## 상세보기 링크
        mizuno_detailEl = mizuno.select(".product-name > .name-link")
        mizuno_detail = mizuno_detailEl[0].attrs['href']
        # print(mizuno_detail)
        # ---------------------------------------------------
            
        #### 상세보기 수집 (시작) 

        # --------------mizuno_detail-------------------------------------
            
        driver.get(mizuno_detail)
        time.sleep(1)
        ## 상세보기 HTML            
        detailSoup = BeautifulSoup(driver.page_source)
        # print(detailSoup)

        # 정가,최종금액 + 리뷰 수 추출
        detailEl = detailSoup.select(".pt_product-details")
        # print(detailEl)
        for detail in detailEl :
            standardPriceEl =detail.select("#product-content > .product-price > span ")
            if len(standardPriceEl) ==1 :
                LastPrice = standardPriceEl[0].text.strip()
                standardPrice = LastPrice
                
                # print(LastPrice)      
            elif len(standardPriceEl) == 3 :
                standardPrice = standardPriceEl[0].text.strip()
                LastPrice = standardPriceEl[1].text.strip()
                
            reviewEl = detail.select(".naverReview > h2 > span")
            review = reviewEl[0].text.strip()
            # print(review)
            
            # ---------------------------------------------------
            
            #### 상세보기 수집 (끝) 

            # --------------mizuno_detail-------------------------------------
            
            ## 수집결과를 딕셔너리에 저장
            result_dict = {"상품이름":mizuno_name,"정가":standardPrice,
                           "최종가격":LastPrice,"리뷰수": review,"이미지링크": mizuno_image}
            ## 최종 결과 리스트에 추가
            mizuno_result.append(result_dict)
df = DataFrame(mizuno_result)


페이지: 1
--------------------------------------------------
페이지: 2
--------------------------------------------------
페이지: 3
--------------------------------------------------
페이지: 4
--------------------------------------------------
페이지: 5
--------------------------------------------------


In [79]:
## 미즈노 상품정보 데이터
df_mizuno= df
df_mizuno

,상품이름,정가,최종가격,리뷰수,이미지링크
0,WAVE REBELLION PRO_J1GC2317,"239,000원","239,000원",(0),https://kor.mizuno.com/dw/image/v2/BBVP_PRD/on...
1,MORELIA SALA JAPAN TF_Q1GB2100,"219,000원","197,100원",(61),https://kor.mizuno.com/dw/image/v2/BBVP_PRD/on...
2,MIZUNO ΑLPHA JAPAN_P1GA2360,"309,000원","278,100원",(32),https://kor.mizuno.com/dw/image/v2/BBVP_PRD/on...
3,MIZUNO ΑLPHA JAPAN AG_P1GA2361,"289,000원","260,100원",(8),https://kor.mizuno.com/dw/image/v2/BBVP_PRD/on...
4,WAVE RIDER 10_D1GA2104,"159,000원","159,000원",(7),https://kor.mizuno.com/dw/image/v2/BBVP_PRD/on...
5,MIZUNO ALPHA SR4 JAPAN_P1GA2399,"329,000원","296,100원",(3),https://kor.mizuno.com/dw/image/v2/BBVP_PRD/on...
6,MORELIA NEO III Β SR4 JAPAN_P1GA2398,"359,000원","323,100원",(2),https://kor.mizuno.com/dw/image/v2/BBVP_PRD/on...
7,MORELIA NEO III SR4 JAPAN_P1GA2397,"329,000원","296,100원",(1),https://kor.mizuno.com/dw/image/v2/BBVP_PRD/on...
8,GRAPHIC T-SHIRTS_32YA3156,"59,000원","59,000원",(5),https://kor.mizuno.com/dw/image/v2/BBVP_PRD/on...
9,GRAPHIC T-SHIRTS_32YA3158,"59,000원","59,000원",(5),https://kor.mizuno.com/dw/image/v2/BBVP_PRD/on...


In [60]:
df_mizuno= df
df_mizuno

## xlsx 파일로 저장 

df_mizuno.to_excel("미즈노_상품정보.xlsx",index=False)
